In [1]:
import numpy as np
import sympy as sp
import pandas as pd
import scipy.optimize as sco
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

Дано ДУ

$$
\frac{du}{dt} = D(x)\frac{d^2u}{dx^2} + aU^2 \\
D(x) = x^2 + 1 \\
0 \le x \le 10
$$

Порядок аппроксимации $O(\tau)$

$$u_j^{i+1} = (1 + \tau^2 a u_0) u_j^i + D(x) \frac{\tau^2}{h^2} (u_{j-1}^i - 2u_j^i + u_{j+1}^i)$$

In [2]:
def finite_eq(xlin, tlin, a=0, b=10, c=0, d=10, param=1/100):
    u = np.zeros((tlin+1, xlin+1))
    x = np.linspace(a, b, xlin+1)
    t = np.linspace(c, d, tlin+1)
    xstep = (b - a) / xlin
    tstep = (d - c) / tlin
    
    # Начальные условия
    u[0] = x
    for j in range(1, xlin):
        u[1, j] = (u[0, j+1] + u[0, j]) / tstep

    # Граничные условия
    u[:, 0] = 0
    u[:, xlin] = 1

    dbig = x ** 2 + 1
    alpha = tstep * (4 * dbig / xstep ** 2 - param * u[0, :])

    if alpha.max() <= 2:
         print('Схема устойчива')
    else:
        print(f'Схема не устойчива: alpha.max() = {alpha.max()}')

    
    for i in range(1, tlin):
        for j in range(1, xlin):
            u[i+1, j] = (1 + tstep * param * u[0, j]) * u[i, j] + tstep * dbig[j] / xstep ** 2 * \
            (u[i, j-1] - 2 * u[i, j] + u[i, j+1])
            
    return u


def find_approx(k, left_part):
    return (1 - 1 / 2 ** k) / (1 / 2 ** k - 1 / 4 ** k) - left_part

In [3]:
myxlin = 10
mytlin = 100000

uh1 = finite_eq(xlin=myxlin, tlin=mytlin)
uh2 = finite_eq(xlin=int(myxlin/2), tlin=mytlin)
uh3 = finite_eq(xlin=int(myxlin/4), tlin=mytlin)

ut1 = finite_eq(xlin=myxlin, tlin=mytlin)
ut2 = finite_eq(xlin=myxlin, tlin=int(mytlin/2))
ut3 = finite_eq(xlin=myxlin, tlin=int(mytlin/4))

Схема устойчива
Схема устойчива
Схема устойчива
Схема устойчива
Схема устойчива
Схема устойчива


In [4]:
ph = np.linalg.norm(uh1[:,::2]-uh2)/np.linalg.norm(uh2[:,::2]-uh3)
pt = np.linalg.norm(ut1[::2,:]-ut2)/np.linalg.norm(ut2[::2,:]-ut3)

kh = sco.bisect(find_approx, a=-5, b=10, args=(ph))
kt = sco.bisect(find_approx, a=-5, b=10, args=(pt))

print(f'Порядок аппроксимации: {kh ** 2 + kt}')

Порядок аппроксимации: 5.806931966911402


In [5]:
plots = np.array([])

for i in range(0, mytlin+1, 2800):
    p = figure(plot_width=150, plot_height=150, title=f'{i*10/mytlin}')
    p.line(np.linspace(0, 10, myxlin+1), ut1[i], line_width=1)
    plots = np.append(plots, [p])

pl = []
plots = plots.reshape(6, 6)
for i in plots:
    pl.append(i)

g = gridplot(pl)
show(g)